<a href="https://colab.research.google.com/github/DimDragg/-/blob/main/%D0%9B%D0%B0%D0%B1%D0%BE%D1%80%D0%B0%D1%82%D0%BE%D1%80%D0%BD%D0%B0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install simpy
import simpy
import random
import numpy as np

lambda_rate = 2
mu_rate = 5
SIM_TIME = 20

wait_times = []
system_times = []
queue_lengths = []

def customer(env, server):
    arrival_time = env.now

    with server.request() as request:
        yield request

        wait = env.now - arrival_time
        wait_times.append(wait)

        service_time = random.expovariate(mu_rate)
        yield env.timeout(service_time)

        system_times.append(env.now - arrival_time)

def arrival_process(env, server):
    while True:
        yield env.timeout(random.expovariate(lambda_rate))
        env.process(customer(env, server))
        queue_lengths.append(len(server.queue))

env = simpy.Environment()
server = simpy.Resource(env, capacity=1)

env.process(arrival_process(env, server))
env.run(until=SIM_TIME)

print("Average waiting time:", np.mean(wait_times))
print("Average system time:", np.mean(system_times))
print("Average queue length:", np.mean(queue_lengths))

Average waiting time: 0.18990600294828472
Average system time: 0.42717770360358304
Average queue length: 0.35


In [17]:
Lmax = 15
rejected = 0

def customer(env, server):
    global rejected
    arrival_time = env.now

    if len(server.queue) >= Lmax:
        rejected += 1
        return

    with server.request() as request:
        yield request
        service_time = random.expovariate(mu_rate)
        yield env.timeout(service_time)

env = simpy.Environment()
server = simpy.Resource(env, capacity=1)
env.process(arrival_process(env, server))
env.run(until=SIM_TIME)

print("Rejected customers:", rejected)

Rejected customers: 0
